In [1]:
import pandas as pd

from functions import join_stocks_crypto, run_clustering_model

c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\tslearn\bases\bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [2]:
kshape_results = pd.read_csv('new_balance_silhouette/kshape.csv')
results = kshape_results

In [3]:
from sklearn.preprocessing import MinMaxScaler
results[['silhouette_norm']] = MinMaxScaler().fit_transform(results[['silhouette_score']])
results['delta_norm'] = 1 - MinMaxScaler().fit_transform(results[['min_max_delta']])
results['total_score'] = (results['silhouette_norm'] + results['delta_norm']) / 2

best_configs = results.loc[results.groupby('method')['total_score'].idxmax()]
best_configs

,Unnamed: 0,clusters,silhouette_score,method,linkage,return_mode,window_size,df_mode,min_per_cluster,max_per_cluster,min_max_delta,silhouette_norm,delta_norm,total_score
68,2,6,0.125491,kshape,not_applicable,geometric,14,full,0.1351,0.2027,0.0676,0.871155,1.0,0.935577


In [4]:
df_all_stocks = pd.read_csv('stocks_data_filled.csv',index_col='Date')
cryptos_df = pd.read_csv('cryptos_data_new.csv', index_col='timestamp')
joined_df = join_stocks_crypto(cryptos_df, df_all_stocks, mode = 'stocks_left')

cryptos_list = list(cryptos_df.columns)

In [5]:
df = joined_df.rolling(window=14, center=True).mean()
labels, tickers_with_labels_kshape, _, _ = run_clustering_model(df, n_clus=6, model_name='kshape', linkage=None, return_mode='geometric', n_init=3)

c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\pandas\core\internals\blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


In [7]:
#for ticker in df_all_stocks.columns
import yfinance as yf     
ticker = yf.Ticker("AAPL")

# Get basic info
info = ticker.fast_info['exchange']

YFRateLimitError: Too Many Requests. Rate limited. Try after a while.

In [18]:
from functions import get_tickers_stocks, get_close_prices, double_listed_stocks
import pandas as pd




#GET THE STOCKS

us_exchanges = ['NMS', 'NYQ', 'NGM']
eu_exchanges = ['PAR', 'FRA', 'LSE', 'AMS']
asia_exchanges = ['SHH', 'JPX', 'HKG']

selected_exchanges = us_exchanges + eu_exchanges + asia_exchanges

full_selected_stocks = {}
df_all_stocks = pd.DataFrame()
for exchange in selected_exchanges:
    print(f'Extracting from {exchange}')
    exchanges = [exchange]
    selected_stocks_dict, ticker_list = get_tickers_stocks(50000, exchanges, 20)

    full_selected_stocks.update(selected_stocks_dict)

    if len(ticker_list) > 0: 
        print('YES')
        df = get_close_prices(ticker_list, period = 2, start = '2022-01-01')
        df_all_stocks = pd.concat([df_all_stocks, df], axis=1)

doubly_listed_tickers = double_listed_stocks(full_selected_stocks)

for ticker_to_drop in doubly_listed_tickers:
    try:
        df_all_stocks = df_all_stocks.drop(columns=[ticker_to_drop])
    except:
        pass

df_all_stocks = df_all_stocks.ffill() #ffill again after concatenating the tickers



Extracting from NMS


YFRateLimitError: Too Many Requests. Rate limited. Try after a while.